# Whatsapp - Chatbot

## Setup

In [26]:
import os, sys, platform
sys.path.append(os.path.abspath('../src'))
if platform.system() == 'Windows':
    os.environ['COMSPEC'] = 'powershell'


## Processar

### Whatsapp - WebDriver

In [27]:
from modules.webdriver.firefox import WebDriverFirefox

webdriver = WebDriverFirefox(False, 3).webdriver


### Whatsapp

In [133]:
import time
from typing import Dict, List


class WhatsAppWeb:
    
    def __init__(self, webdriver):
        self.tempo_espera = 1
        self.webdriver = webdriver
        
    def acessar_site(self):
        self.webdriver.get('https://web.whatsapp.com/')

    def listar_contatos(self) -> List[str]:
        self.__comecar()
        lista_elementos = self.webdriver.find_elements_by_xpath('//span[@dir and @title]')
        lista_contatos_ = []
        for elemento in lista_elementos:
            nome = elemento.text
            if nome not in lista_contatos_:
                lista_contatos_.append(nome)
        return sorted(lista_contatos_)

    def listar_contatos_e_telefones(self) -> List[Dict[str, str]]:
        lista_dados = []
        lista_contatos = self.listar_contatos()
        for nome in lista_contatos:
            self.__comecar()
            dado = self.contato_e_telefone(nome)
            lista_dados.append(dado)
        return lista_dados
    
    def contato_e_telefone(self, nome: str) -> Dict[str, str]:
        try:
            dado = {}
            dado['nome'] = nome
            self.clicar_contato(nome)
            time.sleep(self.tempo_espera)
            elemento = self.webdriver.find_elements_by_xpath('//div[@class="PVMjB"]').pop()
            elemento.click()
            elemento = self.webdriver.find_elements_by_xpath('//*[@title="Dados do contato"]').pop()
            elemento.click()
            elemento = self.webdriver.find_elements_by_xpath(f'//span[@class="_1X4JR"]').pop(3)
            time.sleep(self.tempo_espera)
            dado['telefone'] = elemento.text
            self.webdriver.find_element_by_xpath(f'//button[@class="t4a8o"]')
        except:
            ...    
        return dado
    
    def clicar_contato(self, nome: str):
        elemento = self.webdriver.find_element_by_xpath(f'//span[@dir and @title="{nome}"]')
        if elemento:
            elemento.click()
        return elemento

    def enviar_mensagem(self, nome: str, mensagem: str):
        self.__comecar()
        mensagem = mensagem.strip()
        if not mensagem:
            return
        self.clicar_contato(nome)
        time.sleep(self.tempo_espera)
        grupo_1 = self.webdriver.find_element_by_xpath('//footer[@class="_2vJ01"]')
        time.sleep(1)
        grupo_2 = grupo_1.find_element_by_xpath('//div[@class="_3uMse"]')
        grupo_2.click()
        time.sleep(self.tempo_espera)
        grupo_3 = grupo_2.find_element_by_class_name('_3FRCZ')
        for caracter in mensagem:
            grupo_3.send_keys(caracter)
        botao_enviar = grupo_1.find_element_by_xpath('//div[@class="_1JNuk"]/button')
        time.sleep(self.tempo_espera)
        botao_enviar.click()
        
    def __comecar(self):
        self.webdriver.find_elements_by_xpath('//div[@class="_1MXsz"]').pop().click()
        painel_lateral = self.webdriver.find_element_by_id('pane-side')
        self.webdriver.execute_script("return arguments[0].scrollIntoView(true);", painel_lateral)


### Comandos

#### Listar Contatos

In [125]:
whatsapp = WhatsAppWeb(webdriver)
whatsapp.listar_contatos()


['Alessandra Cassia Da Silva',
 'Anne Carolinne',
 'Apê11',
 'Barriguinha mole',
 'Cristina (Apê 11)',
 'Geraldo Lima',
 'Gislaine Coelho',
 'Luana (Apê11)',
 'Luiz Anselmo da Silva',
 'Meetup Tec',
 'Nerdices (política)',
 'Rita Silva',
 'Rosemeire',
 'Silmara Panhota',
 'Valdir Miranda']

#### Enviar Mensagem

In [51]:
whatsapp = WhatsAppWeb(webdriver)
# whatsapp.acessar_site()
# contatos = whatsapp.listar_contatos()
# whatsapp.clicar_contato('Anne Carolinne')
# whatsapp.enviar_mensagem('Anne Carolinne', 'Oi')
# whatsapp.contato_e_telefone('Anne Carolinne')
whatsapp.enviar_mensagem('Anne Carolinne', 'WhatsApp é um aplicativo multiplataforma de mensagens instantâneas e chamadas de voz para smartphones. Além de mensagens de texto, os usuários podem enviar imagens, vídeos e documentos em PDF, além de fazer ligações grátis por meio de uma conexão com a internet')


In [15]:
for caracter in 'TE AMO MINHA CREMOSA':
    whatsapp.enviar_mensagem('Anne Carolinne', caracter)

In [36]:
contatos

['Anne Carolinne',
 'Vinícius (Barbearia)',
 'Silmara Panhota',
 'Gislaine Coelho',
 'Alessandra Cassia Da Silva',
 'Gislaine Coelho',
 'Valdir Miranda',
 'Luana (Apê11)',
 'Luiz Anselmo da Silva',
 'Gislaine Coelho',
 'Cristina (Apê 11)',
 'Barriguinha mole',
 'Rita Silva',
 'Nerdices (política)',
 'Meetup Tec',
 'Rita Silva']

### Mensagens em massa

In [56]:
lista_contatos = [
 'Anne Carolinne',
 'Rosemeire',
 'Silmara Panhota',
 'Rita Silva',
 'Luiz Anselmo da Silva'
]

whatsapp = WhatsAppWeb(webdriver)

for contato in lista_contatos:
    whatsapp.enviar_mensagem(contato, 'Estou testando umas paradas no meu computador!')


### Obter dados do contato

In [134]:
whatsapp = WhatsAppWeb(webdriver)
# whatsapp.listar_contatos()
# whatsapp.enviar_mensagem('Anne Carolinne', 'Estou testando umas paradas no meu computador!')
# whatsapp.contato_e_telefone('Rita Silva')
print(repr(whatsapp.listar_contatos_e_telefones()))

[{'nome': '+55 11 96922-3615', 'telefone': '+55 11 96922-3615'}, {'nome': 'Agendamentos só chamar!'}, {'nome': 'Alessandra Cassia Da Silva', 'telefone': '+55 11 99265-8248'}, {'nome': 'Barriguinha mole'}, {'nome': 'Cecilia Garcia Miranda'}, {'nome': 'Cristina (Apê 11)', 'telefone': '+55 11 98199-2975'}, {'nome': 'Geraldo Lima', 'telefone': '+55 11 98163-1335'}, {'nome': 'Gislaine Coelho', 'telefone': '+55 11 96638-1918'}, {'nome': 'Leo (Apê11)', 'telefone': '+55 11 97504-2027'}, {'nome': 'Luana (Apê11)', 'telefone': '+55 11 98776-7177'}, {'nome': 'Marcia (Allegrini)'}, {'nome': 'Meetup Tec'}, {'nome': 'Nerdices (política)'}, {'nome': 'Salete Panhota'}, {'nome': 'Valdir Miranda', 'telefone': '+55 11 97395-0172'}, {'nome': 'Vinícius (Barbearia)', 'telefone': 'Agendamentos só chamar!'}]


In [82]:
elemento = webdriver.find_elements_by_xpath('//div[@class="PVMjB"]').pop()
elemento.attribute('innerHtml')
# elemento.click()
# elemento = webdriver.find_elements_by_xpath('//*[@title="Dados do contato"]').pop()
# elemento.click()
# elemento = webdriver.find_elements_by_xpath(f'//span[@class="_1X4JR"]').pop(3)


AttributeError: 'FirefoxWebElement' object has no attribute 'attribute'